# Otimização de Código em Julia

Quando queremos realizar computação de alto desempenho, o tempo necessário para realizar um trabalho computacional é crítico. Quando precisamos lidar com uma grande quantidade de processamento, o tempo gasto em cada operação não é trivial. Portanto, mesmo que o código esteja paralelizado, o código serial precisa executar tão rápido quanto possível.

**nota adicional**: Aos realizar os testes no Pluto, houve inconsistências que até o presente momento não consigo explicar. No entanto, ao usar o jupyter e o REPL, obtive resultados consistentes com o esperado. Por favor, tenha isso em mente sempre que forem usar o Pluto para auferir desempenho.

## Pacotes

In [1]:
using BenchmarkTools

In [25]:
using StaticArrays

┌ Info: Precompiling StaticArrays [90137ffa-7385-5640-81b9-e52037218182]
└ @ Base loading.jl:1260


## Memória e cache-awareness

Quando utilizamos um código, precisamos prestar atenção em como a memória é acessada. Quando acessamos matrizes, por exemplo, na verdade o que temos é um açúcar sintático que permite que identifiquemos as posições por índices; na memória, as posições estão sempre alocados unidimensionalmente.



In [2]:
A = [i + (j-1)*3 for i in 1:3, j in 1:3]

3×3 Array{Int64,2}:
 1  4  7
 2  5  8
 3  6  9

In [4]:
B = Array{Int64, 2}(undef, 3, 3)
    for i in 1:3, j in 1:3
        B[i,j] = i + (j-1)*3
    end
@show B

B = [1 4 7; 2 5 8; 3 6 9]


3×3 Array{Int64,2}:
 1  4  7
 2  5  8
 3  6  9

## Julia é Column-major
![figure](https://eli.thegreenplace.net/images/2015/column-major-2D.png)
E como Julia é Column major, isso significa que qualquer iteração que tenha colunas no loop maior terá melhor eficiência no uso da cache

In [ ]:
S = rand(100,100)
T = rand(100,100)
U = rand(100,100)

In [12]:
function inner_rows!(C, A, B)
    for i in 1:100, j in 1:100
        C[i,j] = A[i,j] + B[i,j]
    end
end

In [13]:
function inner_cols!(C, A, B)
    for j in 1:100, i in 1:100
        C[i,j] = A[i,j] + B[i, j]
    end
end

In [14]:
inner_rows!(U, T, S)
inner_cols!(U, T, S)

In [15]:
@btime inner_rows!(U, T, S)

  17.305 μs (0 allocations: 0 bytes)


In [16]:
@btime inner_cols!(U, T, S)

  10.666 μs (0 allocations: 0 bytes)


## Stack, Heap e mutabilidade

Objetos em Julia podem ter alocação estática ou dinâmica. Variáveis com alocação estática podem ser colocados na stack para acesso rápido, pois seu tamanho é conhecido em tempo de compilação. Variáveis de tamanho dinâmico, por sua vez,  não tem seu tamanho definido em tempo de compilação, o que significa que seu valor tem que ser mantido por um ponteiro àquela região de memória.
A stack possui acesso rápido, porém tem tamanho limitado. Um objeto com tamanho dinâmico provê uma estrutura de dados flexível, mas precisa ser acessado por indireção de ponteiro.
Para ter uma boa performance, é necessário saber onde as estruturas de dados habitam e como o compilador está acessando aquela estrutura/objeto.

In [17]:
a = 50
typeof(a)

Int64

In [18]:
typeof(S)

Array{Float64,2}

Julia trabalha com Arrays de tamanho dinâmico; portanto o compilador não infere o tamanho de um Array em tempo de compilação.

In [21]:
function inner_alloc!(C,A,B)
  for j in 1:100, i in 1:100
    val = [A[i,j] + B[i,j]]
    C[i,j] = val[1]
  end
end
inner_alloc!(U,S,T)

In [48]:
function inner_noalloc!(C,A,B)
    for j in 1:100, i in 1:100
        val = A[i,j] + B[i,j]
        C[i,j] = val
    end
end
inner_noalloc!(U,S,T)

In [28]:
function inner_alloc_static!(C,A,B)
  for j in 1:100, i in 1:100
    val = @SVector [A[i,j] + B[i,j]]
    C[i,j] = val[1]
  end
end
inner_alloc_static!(U,S,T)

In [29]:
@btime inner_alloc!(U,S,T)

  285.680 μs (10000 allocations: 937.50 KiB)


In [50]:
@btime inner_noalloc!(U,S,T)

  17.122 μs (0 allocations: 0 bytes)


In [31]:
@btime inner_alloc_static!(U,S,T)

  10.632 μs (0 allocations: 0 bytes)


De modo geral, a alocação na stack se destina a objetos pequenos cujo tamanho é definido em tempo de compilação. Para objetos grandes, é preferível o uso da heap.

## Mutação
Uma vez que alocar é custoso em termos de performance, sempre que possível, é preferível utilizar espaços de endereçamento já alocados para realizar a computação. Deste modo, é possível ter funções com zero alocações.

In [75]:
function inner_alloc(A,B)
    # 10*(A + B)
    temp = similar(A)
    for j in 1:100, i in 1:100
        C[i,j] = (A[i,j] + B[i,j]) * 10
    end
end
inner_alloc(S,T)

In [76]:
@btime inner_alloc(S,T)

  20.719 μs (2 allocations: 78.20 KiB)


In [ ]:
function inner_noalloc!(C,A,B) # usando um cache Array C já alocado e mutamos
    
    